In [1]:
import torchvision.utils as vutils
:ls ./

SyntaxError: invalid syntax (<ipython-input-1-ae0b918b78d5>, line 2)

In [2]:
import torchvision
image_size = 64
batch_size = 128
dataroot = 'celeba/img_align_celeba'
num_workers = 2
dataset = torchvision.datasets.ImageFolder(root=dataroot, transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workders=num_workders)
real_batch = next(iter(dataloader))
plt.figure(figsize=(16, 16))
plt.axis=('off')
plt.title('Training Images')
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True)))

NameError: name 'torchvision' is not defined

In [3]:
def weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        
nz = 100
ngf = 64
nc = 3
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size: ngf * 8, 4, 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size: ngf * 4, 8, 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size: ngf * 2, 16, 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size: ngf, 32, 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh(),
            # state size: nc, 64, 64
        )
    def forward(self, x):
        return self.main(x)

G = Generator().to(device)
G.apply(weight_init)

NameError: name 'nn' is not defined

In [4]:
ndf = 64
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            #state size: nc, 64, 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=True),
            nn.LeakReLU(0.2, inplace=True),
            # state size: ndf, 32, 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace = True),
            # state size: ndf * 2, 16, 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace = True),
            # state size: ndf * 4, 8, 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace = True),
            # state size: ndf * 8, 4, 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.main(x)
    
D = Discriminator().to(device)
D.apply(weight_init)
print(D)

NameError: name 'nn' is not defined

In [5]:
loss_fn = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr = 0.002, betas=[0.5, 0.999])
g_optimizer = torch.optim.Adam(G.parameters(), lr = 0.002, betas=[0.5, 0.999])


total_steps = len(dataloader)
num_epochs = 5
for epoch in range(num_epochs):
    for i, (image, _) in enumerate(dataloader):
        batch_size = images.shape[0]
        iamges = image.to(device)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        outputs = D(images)
        d_loss_real = loss_fn(outputs, real_labels)
        real_score = outputs # 对于D来说，越大越好
        
        # 开始生成fake image
        z = torch.randn(batch_size, nz, 1, 1).to(device)# latent variable
        fake_images = G(z)
        outputs= D(fake_images.detach())
        d_loss_fake = loss_fn(outputs, fake_labels)
        fake_score = outputs # 对于D来说，越小越好
        
        # 开始优化 discriminator
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 开始优化generator
        outputs = D(fake_images)
        g_loss = loss_fn(outputs, real_labels)
        
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        
        g_loss.backward()
        g_optimizer.step()
        
        if i % 200 == 0:
            print('Epoch [{}/{}], step[{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x):{:.2f}, D(G(z)): {:.2f}'
                 .format(epoch, num_epochs, i, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(),
                        fake_score.mean().item()))

NameError: name 'nn' is not defined

In [ ]:
total_steps = len(dataloader)
num_epochs = 5
for epoch in range(num_epochs):
     for i, data in enumerate(dataloader):
            D.zero_grad()
            real_images = data[0].to(device)
            b_size = real_images.size(0)
            real_label = torch.ones(b_size).to(device)
            output = D(real_images).view(-1)
            
            real_loss = loss_fn(output, lable)
            real_loss.backward()
            D_x = output.mean().item()
            
            # 生成图片
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            fake_images = G(noise)
            fake_label = torch.zeros(b_size).to(device)
            output = D(fake_images.detach()).view(-1)
            fake_loss = loss_fn(output, fake_label)
            # fake_loss.backward()
            D_G_z1 = output.mean().item()
            loss_D = real_loss + fake_loss
            d_optimizer.step()
            
            #训练generator
            G.zero_grad()
            lable.fill_(1)
            output = D(fake_images).view(-1)
            loss_G = loss_fn(output, real_label)
            loss_G.backward()
            D_G_z2 = output.mean().item()
            g_optimizer.step()
            
            if i % 50 == 0:
                print('[{}/{}] [{}/{}] Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(x)): {:.4f}'
                 .format(epoch, num_epochs, i, len(dataloader), loss_D.item(), loss_G.item())